# Exportation des 100 plus grosses erreurs

On veut exporter dans un tableau excel, nos 100 plus grosses erreurs afin de pouvoir analyser nos données plus simplement.


In [26]:
from bigml.api import BigML
import numpy as np
from pandas import DataFrame, read_csv

prediction=read_csv('./BatchPrediction/val_predictions.csv', index_col=0)
#on change le nom de la colonne SeriousDlqin2yrs, on peut aussi le faire avec .rename

prediction.insert(4,"Type_Error","")
prediction

,SeriousDlqin2yrs,Prediction,0 Proba,1 Proba,Type_Error
field1,,,,,
99622,1,1,0.28041,0.71959,
86498,1,1,0.29142,0.70858,
97370,1,1,0.29166,0.70834,
105591,1,1,0.29307,0.70693,
146045,0,1,0.29596,0.70404,
...,...,...,...,...,...
6164,0,0,0.99845,0.00156,
29862,0,0,0.99845,0.00156,
50195,0,0,0.99845,0.00156,


In [29]:
def fctError(row):
    if row['SeriousDlqin2yrs']==1 and row['Prediction']==0:
        row['Type_Error']='FN'
    if row['SeriousDlqin2yrs']==0 and row['Prediction']==1:
        row['Type_Error']='FP'
    if row['SeriousDlqin2yrs']==1 and row['Prediction']==1:
        row['Type_Error']='TP'
    if row['SeriousDlqin2yrs']==0 and row['Prediction']==0:
        row['Type_Error']='TN'
    return row

In [30]:
prediction['Type_Error']=prediction[['Type_Error','SeriousDlqin2yrs','Prediction']].apply(fctError, axis=1)

On peut dénombrer les FN, FP, TP et TN grâce à la fonction .value_counts()

In [32]:
number_error = prediction['Type_Error'].value_counts()
print(number_error)

TN    27748
FN     1613
TP      387
FP      252
Name: Type_Error, dtype: int64


On tri selon les colonnes 'Type_Error' et '0 Proba' de manière à avoir les FN en premier et décroissante selon 0 Proba, afin de pouvoir sélectionner et enregistrer en csv les 100 plus grosses erreurs.

In [47]:
sorted_prediction = prediction.sort_values(by=['Type_Error', '0 Proba'], ascending=["FN", False])

In [50]:
sorted_prediction[0:99].to_csv('top100_fn_error.csv')